In [1]:
✅Veri toplama: Kredi başvuru formlarından verilerin toplanması gerekir. Bu veriler, kişisel bilgiler (yaş, cinsiyet, eğitim, iş, gelir vb.), kredi geçmişi (örneğin, geçmiş kredi ödemeleri, açık krediler, borçlar vb.) ve diğer önemli faktörleri (örneğin, ev sahipliği, araba sahipliği, evli mi vb.) içerebilir.

▶️Veri temizleme: Toplanan verilerin doğruluğunu ve tamamını kontrol etmek gerekir. Eksik veriler tamamlanmalı veya veri setinden çıkarılmalıdır.

Veri ön işleme: Veri setindeki verilerin sayısal verilere dönüştürülmesi gerekir. Bu, etiketlenmemiş verileri sınıflandırma modelleri tarafından kullanılabilir hale getirecektir.

Modellerin eğitilmesi: Veri seti, eğitim ve test verilerine ayrılmalıdır. Eğitim verileri kullanılarak birkaç sınıflandırma algoritması (örneğin, K-NN, Random Forest, Naive Bayes vb.) eğitilmelidir.

Modellerin değerlendirilmesi: Eğitilen modeller, test verileri kullanılarak değerlendirilmelidir. Değerlendirme metrikleri (örneğin, doğruluk, F1 skoru, ROC eğrisi vb.) kullanılarak modellerin performansları karşılaştırılmalıdır.

En iyi modelin seçimi: En yüksek performans gösteren model seçilmelidir.

Uygulama: Seçilen model, gerçek veriler üzerinde uygulanmalı ve kredi başvurularının riskli veya güvenli olarak sınıflandırılması yapılmalıdır.

In [121]:
import pandas as pd
import numpy as np
import scipy.stats as stats 

# Data

In [2]:
df_original_train = pd.read_csv('/Users/beyazituysal/Downloads/archive/train.csv',low_memory=False)
df_original_test = pd.read_csv('/Users/beyazituysal/Downloads/archive/test.csv')

### Understanding Data

In [3]:
df_train = df_original_train.copy()
df_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [4]:
df_test = df_original_test.copy()
df_test.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [5]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Monthly_Inhand_Salary,84998.0,4194.170850,3183.686167,303.645417,1625.568229,3093.745000,5957.448333,15204.633333
Num_Bank_Accounts,100000.0,17.091280,117.404834,-1.000000,3.000000,6.000000,7.000000,1798.000000
Num_Credit_Card,100000.0,22.474430,129.057410,0.000000,4.000000,5.000000,7.000000,1499.000000
Interest_Rate,100000.0,72.466040,466.422621,1.000000,8.000000,13.000000,20.000000,5797.000000
Delay_from_due_date,100000.0,21.068780,14.860104,-5.000000,10.000000,18.000000,28.000000,67.000000
Num_Credit_Inquiries,98035.0,27.754251,193.177339,0.000000,3.000000,6.000000,9.000000,2597.000000
Credit_Utilization_Ratio,100000.0,32.285173,5.116875,20.000000,28.052567,32.305784,36.496663,50.000000
Total_EMI_per_month,100000.0,1403.118217,8306.041270,0.000000,30.306660,69.249473,161.224249,82331.000000


In [6]:
df_train.shape, df_test.shape

((100000, 28), (50000, 27))

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        50000 non-null  object 
 1   Customer_ID               50000 non-null  object 
 2   Month                     50000 non-null  object 
 3   Name                      44985 non-null  object 
 4   Age                       50000 non-null  object 
 5   SSN                       50000 non-null  object 
 6   Occupation                50000 non-null  object 
 7   Annual_Income             50000 non-null  object 
 8   Monthly_Inhand_Salary     42502 non-null  float64
 9   Num_Bank_Accounts         50000 non-null  int64  
 10  Num_Credit_Card           50000 non-null  int64  
 11  Interest_Rate             50000 non-null  int64  
 12  Num_of_Loan               50000 non-null  object 
 13  Type_of_Loan              44296 non-null  object 
 14  Delay_

In [9]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Monthly_Inhand_Salary,84998.0,4194.170850,3183.686167,303.645417,1625.568229,3093.745000,5957.448333,15204.633333
Num_Bank_Accounts,100000.0,17.091280,117.404834,-1.000000,3.000000,6.000000,7.000000,1798.000000
Num_Credit_Card,100000.0,22.474430,129.057410,0.000000,4.000000,5.000000,7.000000,1499.000000
Interest_Rate,100000.0,72.466040,466.422621,1.000000,8.000000,13.000000,20.000000,5797.000000
Delay_from_due_date,100000.0,21.068780,14.860104,-5.000000,10.000000,18.000000,28.000000,67.000000
Num_Credit_Inquiries,98035.0,27.754251,193.177339,0.000000,3.000000,6.000000,9.000000,2597.000000
Credit_Utilization_Ratio,100000.0,32.285173,5.116875,20.000000,28.052567,32.305784,36.496663,50.000000
Total_EMI_per_month,100000.0,1403.118217,8306.041270,0.000000,30.306660,69.249473,161.224249,82331.000000


In [10]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Monthly_Inhand_Salary,42502.0,4182.004291,3174.109304,303.645417,1625.188333,3086.305000,5934.189094,15204.633333
Num_Bank_Accounts,50000.0,16.838260,116.396848,-1.000000,3.000000,6.000000,7.000000,1798.000000
Num_Credit_Card,50000.0,22.921480,129.314804,0.000000,4.000000,5.000000,7.000000,1499.000000
Interest_Rate,50000.0,68.772640,451.602363,1.000000,8.000000,13.000000,20.000000,5799.000000
Delay_from_due_date,50000.0,21.052640,14.860397,-5.000000,10.000000,18.000000,28.000000,67.000000
Num_Credit_Inquiries,48965.0,30.080200,196.984121,0.000000,4.000000,7.000000,10.000000,2593.000000
Credit_Utilization_Ratio,50000.0,32.279581,5.106238,20.509652,28.061040,32.280390,36.468591,48.540663
Total_EMI_per_month,50000.0,1491.304305,8595.647887,0.000000,32.222388,74.733349,176.157491,82398.000000


In [11]:
 df_train.describe(exclude=np.number).T

,count,unique,top,freq
ID,100000,100000,0x1602,1
Customer_ID,100000,12500,CUS_0xd40,8
Month,100000,8,January,12500
Name,90015,10139,Langep,44
Age,100000,1788,38,2833
SSN,100000,12501,#F%$D@*&8,5572
Occupation,100000,16,_______,7062
Annual_Income,100000,18940,36585.12,16
Num_of_Loan,100000,434,3,14386
Type_of_Loan,88592,6260,Not Specified,1408


In [12]:
df_test.describe(exclude=np.number).T

,count,unique,top,freq
ID,50000,50000,0x160a,1
Customer_ID,50000,12500,CUS_0xd40,4
Month,50000,4,September,12500
Name,44985,10139,Stevex,22
Age,50000,976,39,1493
SSN,50000,12501,#F%$D@*&8,2828
Occupation,50000,16,_______,3438
Annual_Income,50000,16121,109945.32,8
Num_of_Loan,50000,263,2,7173
Type_of_Loan,44296,6260,Not Specified,704


### Missing Values 

In [13]:
df_train['Credit_Score'].isna().sum()

0

In [14]:
(df_train.columns[:-1]!=df_test.columns).sum()

0

In [15]:
#merging data
df = pd.concat([df_train, df_test], ignore_index=True)
df.shape

(150000, 28)

In [16]:
df['Credit_Score'].isna().sum()

50000

In [17]:
df.isnull().mean()*100

ID                           0.000000
Customer_ID                  0.000000
Month                        0.000000
Name                        10.000000
Age                          0.000000
SSN                          0.000000
Occupation                   0.000000
Annual_Income                0.000000
Monthly_Inhand_Salary       15.000000
Num_Bank_Accounts            0.000000
Num_Credit_Card              0.000000
Interest_Rate                0.000000
Num_of_Loan                  0.000000
Type_of_Loan                11.408000
Delay_from_due_date          0.000000
Num_of_Delayed_Payment       7.000000
Changed_Credit_Limit         0.000000
Num_Credit_Inquiries         2.000000
Credit_Mix                   0.000000
Outstanding_Debt             0.000000
Credit_Utilization_Ratio     0.000000
Credit_History_Age           9.000000
Payment_of_Min_Amount        0.000000
Total_EMI_per_month          0.000000
Amount_invested_monthly      4.500000
Payment_Behaviour            0.000000
Monthly_Bala

In [18]:
df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

### Examing the Data

In [19]:
df.select_dtypes('O').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ID                       150000 non-null  object
 1   Customer_ID              150000 non-null  object
 2   Month                    150000 non-null  object
 3   Name                     135000 non-null  object
 4   Age                      150000 non-null  object
 5   SSN                      150000 non-null  object
 6   Occupation               150000 non-null  object
 7   Annual_Income            150000 non-null  object
 8   Num_of_Loan              150000 non-null  object
 9   Type_of_Loan             132888 non-null  object
 10  Num_of_Delayed_Payment   139500 non-null  object
 11  Changed_Credit_Limit     150000 non-null  object
 12  Credit_Mix               150000 non-null  object
 13  Outstanding_Debt         150000 non-null  object
 14  Credit_History_Age  

In [20]:
object_col = df.describe(include='O').columns
object_col

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Num_of_Loan', 'Type_of_Loan',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_History_Age', 'Payment_of_Min_Amount',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [21]:
df = df.applymap(lambda x: x if x is np.NaN or not isinstance(x, str) else str(x).strip('_ ,"')).replace(['', 'nan', '!@9#%8', '#F%$D@*&8'], np.NaN)
df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,NaN,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0x25fe5,CUS_0x8600,December,Sarah McBridec,4975,031-35-0942,Architect,20002.88,1929.906667,10,...,NaN,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343,NaN
149996,0x25fee,CUS_0x942c,September,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066,NaN
149997,0x25fef,CUS_0x942c,October,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,10000,Low_spent_Large_value_payments,349.7263321025098,NaN
149998,0x25ff0,CUS_0x942c,November,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717,NaN


In [22]:
df.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                        15000
Age                             0
SSN                          8400
Occupation                  10500
Annual_Income                   0
Monthly_Inhand_Salary       22500
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                17112
Delay_from_due_date             0
Num_of_Delayed_Payment      10500
Changed_Credit_Limit         3150
Num_Credit_Inquiries         3000
Credit_Mix                  30000
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age          13500
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      6750
Payment_Behaviour           11400
Monthly_Balance              1762
Credit_Score                50000
dtype: int64

## Fixing Data

In [23]:
df.select_dtypes('O').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ID                       150000 non-null  object
 1   Customer_ID              150000 non-null  object
 2   Month                    150000 non-null  object
 3   Name                     135000 non-null  object
 4   Age                      150000 non-null  object
 5   SSN                      141600 non-null  object
 6   Occupation               139500 non-null  object
 7   Annual_Income            150000 non-null  object
 8   Num_of_Loan              150000 non-null  object
 9   Type_of_Loan             132888 non-null  object
 10  Num_of_Delayed_Payment   139500 non-null  object
 11  Changed_Credit_Limit     146850 non-null  object
 12  Credit_Mix               120000 non-null  object
 13  Outstanding_Debt         150000 non-null  object
 14  Credit_History_Age  

##### Column type chancing

In [24]:
df['ID'] = df.ID.apply(lambda x: int(x, 16))

In [25]:
df['Customer_ID'] = df.Customer_ID.apply(lambda x: int(x[4:], 16))

In [26]:
df['Month'] = pd.to_datetime(df.Month, format='%B').dt.month

In [27]:
df['Age'] = df.Age.astype(int)

In [28]:
df['SSN'] = df.SSN.apply(lambda x: x if x is np.NaN else int(str(x).replace('-', ''))).astype(float)

In [29]:
df['Annual_Income'] = df.Annual_Income.astype(float)

In [30]:
df['Num_of_Loan'] = df.Num_of_Loan.astype(int) 

In [31]:
df['Num_of_Delayed_Payment'] = df.Num_of_Delayed_Payment.astype(float)

In [32]:
df['Changed_Credit_Limit'] = df.Changed_Credit_Limit.astype(float)

In [33]:
df['Outstanding_Debt'] = df.Outstanding_Debt.astype(float)

In [34]:
df['Amount_invested_monthly'] = df.Amount_invested_monthly.astype(float)

In [35]:
df['Monthly_Balance'] = df.Monthly_Balance.astype(float)

In [38]:
ya = df['Credit_History_Age']
ya

0          22 Years and 1 Months
1                            NaN
2          22 Years and 3 Months
3          22 Years and 4 Months
4          22 Years and 5 Months
                   ...          
149995                       NaN
149996    31 Years and 11 Months
149997     32 Years and 0 Months
149998     32 Years and 1 Months
149999     32 Years and 2 Months
Name: Credit_History_Age, Length: 150000, dtype: object

In [49]:
def Year_Month_converter(date):
    if pd.notnull(date):
        data = date.split(' ')
        year = int(data[0])
        month = int(data[-2])
      
        return (year*12)+month
    else:
        return date

In [50]:
Year_Month_converter(df['Credit_History_Age'][0])

265

In [52]:
df['Credit_History_Age'] = df.Credit_History_Age.apply(lambda x: Year_Month_converter(x)).astype(float)

In [53]:
df.groupby('Customer_ID')['Credit_History_Age'].apply(list)

Customer_ID
1006     [182.0, 183.0, 184.0, 185.0, 186.0, 187.0, 188...
1007     [346.0, 347.0, 348.0, 349.0, 350.0, nan, 352.0...
1008     [292.0, 293.0, 294.0, nan, 296.0, 297.0, 298.0...
1009     [331.0, 332.0, 333.0, 334.0, 335.0, 336.0, 337...
1011     [179.0, 180.0, nan, 182.0, 183.0, 184.0, 185.0...
                               ...                        
50984    [157.0, 158.0, 159.0, nan, 161.0, 162.0, nan, ...
50990    [70.0, 71.0, 72.0, nan, nan, 75.0, 76.0, 77.0,...
50992    [159.0, 160.0, 161.0, 162.0, 163.0, nan, 165.0...
50996    [206.0, 207.0, 208.0, 209.0, 210.0, 211.0, 212...
50999    [226.0, 227.0, 228.0, 229.0, 230.0, 231.0, 232...
Name: Credit_History_Age, Length: 12500, dtype: object

In [54]:
df.groupby('Customer_ID')['Type_of_Loan'].value_counts(dropna=False)

Customer_ID  Type_of_Loan                                                                                         
1006         Credit-Builder Loan, and Payday Loan                                                                     12
1007         Home Equity Loan, Mortgage Loan, and Student Loan                                                        12
1008         NaN                                                                                                      12
1009         Credit-Builder Loan, Student Loan, Not Specified, and Student Loan                                       12
1011         Personal Loan, Auto Loan, and Auto Loan                                                                  12
                                                                                                                      ..
50984        Home Equity Loan, Mortgage Loan, Payday Loan, Mortgage Loan, Mortgage Loan, and Payday Loan              12
50990        Mortgage Loan, Auto Loan,

In [55]:
df.groupby('Customer_ID')['Type_of_Loan'].apply(list)

Customer_ID
1006     [Credit-Builder Loan, and Payday Loan, Credit-...
1007     [Home Equity Loan, Mortgage Loan, and Student ...
1008     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
1009     [Credit-Builder Loan, Student Loan, Not Specif...
1011     [Personal Loan, Auto Loan, and Auto Loan, Pers...
                               ...                        
50984    [Home Equity Loan, Mortgage Loan, Payday Loan,...
50990    [Mortgage Loan, Auto Loan, and Auto Loan, Mort...
50992    [Student Loan, Credit-Builder Loan, Mortgage L...
50996    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
50999    [Credit-Builder Loan, Credit-Builder Loan, Cre...
Name: Type_of_Loan, Length: 12500, dtype: object

In [88]:
def different_values_column(df_column):
    diff_value = set()
    for i in df_column.dropna():
        values = set(map(str.strip, i.replace('and', ',').split(',')))
        diff_value |= values
        dic = {i-1:v for i,v in enumerate(sorted(diff_value))}
        del dic[-1]# for remove the empty index
    return dic

In [89]:
different_values_column(df['Type_of_Loan'])

{0: 'Auto Loan',
 1: 'Credit-Builder Loan',
 2: 'Debt Consolidation Loan',
 3: 'Home Equity Loan',
 4: 'Mortgage Loan',
 5: 'Not Specified',
 6: 'Payday Loan',
 7: 'Personal Loan',
 8: 'Student Loan'}

In [105]:
def reassign_group_object_nan_values(df, groupby, column, inplace=True):
    if not inplace:
        df = df.copy()
        
    def assign_nans(df, groupby, column):
        # Replace None with np.NaN
        df[column] = df[column].fillna(np.NaN)
        # Fill with local mode
        df[column] = df.groupby(groupby)[column].transform(lambda x: x.fillna(x.mode()[0]))
        
    # Before
    print(f'Before Assigning NaN {column}: {df[column].value_counts(dropna=False).head(1)}')
    a = df.groupby(groupby)[column].apply(list)
    print(f'\nBefore Assigning Example {column}:\n{a.head()}\n')
    
    assign_nans(df, groupby, column)
    
    # After
    print(f'After Assigning NaN {column}: {df[column].value_counts(dropna=False).head(1)}')
    b = df.groupby(groupby)[column].apply(list)
    print(f'\nAfter Assigning Example {column}:\n{b.head()}\n')
    
    if not inplace:
        return df


In [106]:
df.describe(include='O').T

,count,unique,top,freq
Name,149988,10128,Jessicad,72
Occupation,139500,15,Lawyer,9899
Type_of_Loan,132888,6260,Not Specified,2112
Credit_Mix,120000,3,Standard,54858
Payment_of_Min_Amount,150000,3,Yes,78484
Payment_Behaviour,138600,6,Low_spent_Small_value_payments,38207
Credit_Score,100000,3,Standard,53174


##### Name

In [107]:
df['Name'].value_counts(dropna=False).head()

Jessicad          72
Langep            72
Stevex            72
Ronald Groverk    60
Danielz           60
Name: Name, dtype: int64

In [108]:
reassign_group_object_nan_values(df, 'Customer_ID', 'Name')

Before Assigning NaN Name: Jessicad    72
Name: Name, dtype: int64

Before Assigning Example Name:
Customer_ID
1006    [Matthias Blamontb, Matthias Blamontb, Matthia...
1007    [Soyoung Kimu, Soyoung Kimu, Soyoung Kimu, Soy...
1008    [Koht, Koht, Koht, Koht, Koht, Koht, Koht, Koh...
1009    [Edd, Edd, Edd, Edd, Edd, Edd, Edd, Edd, Edd, ...
1011    [Terry Wadeu, Terry Wadeu, Terry Wadeu, Terry ...
Name: Name, dtype: object

After Assigning NaN Name: Jessicad    72
Name: Name, dtype: int64

After Assigning Example Name:
Customer_ID
1006    [Matthias Blamontb, Matthias Blamontb, Matthia...
1007    [Soyoung Kimu, Soyoung Kimu, Soyoung Kimu, Soy...
1008    [Koht, Koht, Koht, Koht, Koht, Koht, Koht, Koh...
1009    [Edd, Edd, Edd, Edd, Edd, Edd, Edd, Edd, Edd, ...
1011    [Terry Wadeu, Terry Wadeu, Terry Wadeu, Terry ...
Name: Name, dtype: object



##### Occupation

In [109]:
df['Occupation'].value_counts(dropna=False)

NaN              10500
Lawyer            9899
Engineer          9562
Architect         9550
Mechanic          9459
Accountant        9404
Scientist         9403
Developer         9381
Media_Manager     9362
Teacher           9318
Entrepreneur      9277
Journalist        9122
Doctor            9114
Manager           8973
Musician          8858
Writer            8818
Name: Occupation, dtype: int64

In [110]:
reassign_group_object_nan_values(df, 'Customer_ID', 'Occupation')

Before Assigning NaN Occupation: NaN    10500
Name: Occupation, dtype: int64

Before Assigning Example Occupation:
Customer_ID
1006    [Journalist, Journalist, Journalist, Journalis...
1007    [Manager, Manager, nan, Manager, Manager, Mana...
1008    [Developer, Developer, Developer, Developer, D...
1009    [Accountant, nan, Accountant, Accountant, Acco...
1011    [Writer, Writer, Writer, Writer, nan, Writer, ...
Name: Occupation, dtype: object

After Assigning NaN Occupation: Lawyer    10644
Name: Occupation, dtype: int64

After Assigning Example Occupation:
Customer_ID
1006    [Journalist, Journalist, Journalist, Journalis...
1007    [Manager, Manager, Manager, Manager, Manager, ...
1008    [Developer, Developer, Developer, Developer, D...
1009    [Accountant, Accountant, Accountant, Accountan...
1011    [Writer, Writer, Writer, Writer, Writer, Write...
Name: Occupation, dtype: object



##### Type_of_Loan

In [111]:
df.groupby('Customer_ID')['Type_of_Loan'].value_counts(dropna=False)

Customer_ID  Type_of_Loan                                                                                         
1006         Credit-Builder Loan, and Payday Loan                                                                     12
1007         Home Equity Loan, Mortgage Loan, and Student Loan                                                        12
1008         NaN                                                                                                      12
1009         Credit-Builder Loan, Student Loan, Not Specified, and Student Loan                                       12
1011         Personal Loan, Auto Loan, and Auto Loan                                                                  12
                                                                                                                      ..
50984        Home Equity Loan, Mortgage Loan, Payday Loan, Mortgage Loan, Mortgage Loan, and Payday Loan              12
50990        Mortgage Loan, Auto Loan,

In [112]:
df['Type_of_Loan'].replace([np.NaN], 'No Data', inplace=True)

##### Credit_Mix

In [113]:
df['Credit_Mix'].value_counts(dropna=False)

Standard    54858
Good        36597
NaN         30000
Bad         28545
Name: Credit_Mix, dtype: int64

In [114]:
reassign_group_object_nan_values(df, 'Customer_ID', 'Credit_Mix')

Before Assigning NaN Credit_Mix: Standard    54858
Name: Credit_Mix, dtype: int64

Before Assigning Example Credit_Mix:
Customer_ID
1006    [Standard, Standard, Standard, Standard, Stand...
1007    [nan, Standard, Standard, Standard, nan, Stand...
1008    [nan, Standard, Standard, nan, Standard, Stand...
1009    [Standard, nan, Standard, Standard, Standard, ...
1011    [nan, nan, Standard, Standard, Standard, Stand...
Name: Credit_Mix, dtype: object

After Assigning NaN Credit_Mix: Standard    68772
Name: Credit_Mix, dtype: int64

After Assigning Example Credit_Mix:
Customer_ID
1006    [Standard, Standard, Standard, Standard, Stand...
1007    [Standard, Standard, Standard, Standard, Stand...
1008    [Standard, Standard, Standard, Standard, Stand...
1009    [Standard, Standard, Standard, Standard, Stand...
1011    [Standard, Standard, Standard, Standard, Stand...
Name: Credit_Mix, dtype: object



##### Payment_of_Min_Amount (There's no need)

In [115]:
df['Payment_of_Min_Amount'].value_counts(dropna=False)

Yes    78484
No     53516
NM     18000
Name: Payment_of_Min_Amount, dtype: int64

##### Payment_Behaviour

In [116]:
df['Payment_Behaviour'].value_counts(dropna=False)

Low_spent_Small_value_payments      38207
High_spent_Medium_value_payments    26462
Low_spent_Medium_value_payments     20698
High_spent_Large_value_payments     20565
High_spent_Small_value_payments     16991
Low_spent_Large_value_payments      15677
NaN                                 11400
Name: Payment_Behaviour, dtype: int64

In [117]:
reassign_group_object_nan_values(df, 'Customer_ID', 'Payment_Behaviour')

Before Assigning NaN Payment_Behaviour: Low_spent_Small_value_payments    38207
Name: Payment_Behaviour, dtype: int64

Before Assigning Example Payment_Behaviour:
Customer_ID
1006    [High_spent_Medium_value_payments, Low_spent_M...
1007    [High_spent_Medium_value_payments, High_spent_...
1008    [High_spent_Small_value_payments, Low_spent_La...
1009    [High_spent_Medium_value_payments, High_spent_...
1011    [High_spent_Small_value_payments, Low_spent_Sm...
Name: Payment_Behaviour, dtype: object

After Assigning NaN Payment_Behaviour: Low_spent_Small_value_payments    42046
Name: Payment_Behaviour, dtype: int64

After Assigning Example Payment_Behaviour:
Customer_ID
1006    [High_spent_Medium_value_payments, Low_spent_M...
1007    [High_spent_Medium_value_payments, High_spent_...
1008    [High_spent_Small_value_payments, Low_spent_La...
1009    [High_spent_Medium_value_payments, High_spent_...
1011    [High_spent_Small_value_payments, Low_spent_Sm...
Name: Payment_Behaviour, dtype: 

### Find Min, Max, Mean

In [118]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,150000.0,8.063350e+04,4.330141e+04,5.634000e+03,4.313375e+04,8.063350e+04,1.181332e+05,1.556330e+05
Customer_ID,150000.0,2.598267e+04,1.434052e+04,1.006000e+03,1.366450e+04,2.577700e+04,3.838500e+04,5.099900e+04
Month,150000.0,6.500000e+00,3.452064e+00,1.000000e+00,3.750000e+00,6.500000e+00,9.250000e+00,1.200000e+01
Age,150000.0,1.103379e+02,6.840668e+02,-5.000000e+02,2.500000e+01,3.300000e+01,4.200000e+01,8.698000e+03
SSN,141600.0,5.005493e+08,2.907135e+08,8.134900e+04,2.453159e+08,5.007606e+08,7.559962e+08,9.999934e+08
Annual_Income,150000.0,1.730552e+05,1.404215e+06,7.005930e+03,1.945549e+04,3.757861e+04,7.279690e+04,2.419806e+07
Monthly_Inhand_Salary,127500.0,4.190115e+03,3.180490e+03,3.036454e+02,1.625266e+03,3.091000e+03,5.948455e+03,1.520463e+04
Num_Bank_Accounts,150000.0,1.700694e+01,1.170695e+02,-1.000000e+00,3.000000e+00,6.000000e+00,7.000000e+00,1.798000e+03
Num_Credit_Card,150000.0,2.262345e+01,1.291430e+02,0.000000e+00,4.000000e+00,5.000000e+00,7.000000e+00,1.499000e+03
Interest_Rate,150000.0,7.123491e+01,4.615372e+02,1.000000e+00,8.000000e+00,1.300000e+01,2.000000e+01,5.799000e+03


In [119]:
df['Customer_ID'].nunique()

12500

In [120]:
# outlier range
def outlier_range(df, column, multiply=1.5):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - iqr * multiply
    upper = q3 + iqr * multiply
    return lower, upper

In [130]:
# Reassign Wrong Values and Show Function
def Reassign_Group_Numeric_Wrong_Values(df, groupby, column, inplace=True):      

    # Identify Wrong values Range
    def get_group_min_max(df, groupby, column):            
        cur = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = cur.apply(lambda x: stats.mode(x, keepdims=True)).apply([min, max])
        return x[0][0], y[0][0]
    
    # Assigning Wrong values
    def make_group_NaN_and_fill_mode(df, groupby, column, inplace=True):
        df_dropped = df[df[column].notna()].groupby(groupby)[column].apply(list)
        x, y = df_dropped.apply(lambda x: stats.mode(x, keepdims=True)).apply([min, max])
        mini, maxi = x[0][0], y[0][0]

        # assign Wrong Values to NaN
        col = df[column].apply(lambda x: np.NaN if ((x<mini)|(x>maxi)) else x)

        # fill with local mode
        mode_by_group = df.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)
        result = col.fillna(mode_by_group)

        # inplace
        if inplace:
            df[column]=result
        else:
            return result
        
    
    # Run      
    if inplace:   
        print("\nExisting Min, Max Values:", df[column].apply([min, max]), sep='\n', end='\n')       
        mini, maxi = get_group_min_max(df, groupby, column)        
        print(f"\nGroupby by {groupby}'s Actual min, max Values:", f'min:\t{mini},\nmax:\t{ maxi}', sep='\n', end='\n')
        
        # Before Assigning Wrong values      
        x = df[column].value_counts(dropna=False).head(1)
        print(f'\nBefore Assigning Min, Max {column}:', [x if x.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        
        a = df.groupby(groupby)[column].apply(list) 
        print(f'\nBefore Assigning Example {column}:\n', *a.head().values, sep='\n', end='\n')
        
        # Assigning
        make_group_NaN_and_fill_mode(df, groupby, column, inplace)
        
        # After Assigning Wrong values
        y = df[column].value_counts(dropna=False).head(1)
        print(f'\nAfter Assigning  Min, Max {column}:', [y if y.index[0] is np.NaN or x.index[0] is None else 'No NaN Value'] , end='\n')
        
        b = df.groupby(groupby)[column].apply(list)
        print(f'\nAfter Assigning Example {column}:\n', *b.head().values, sep='\n', end='\n')
    else:   
        # Show
        return make_group_NaN_and_fill_mode(df, groupby, column, inplace)


In [ ]:
'''TR
Bu fonksiyon, bir veri çerçevesinde belirtilen bir gruplama sütununa göre belirli bir sayısal sütunda yanlış değerleri tespit eder ve bunları grup ortalamaları veya medyanlarıyla doldurur.

Fonksiyon üç adımdan oluşur:

1.get_group_min_max: Verilen gruplama sütununa göre, sütundaki değerlerin modunu alır ve minimum-maximum aralığını hesaplar. Bu, sayısal sütunun olası yanlış değerlerini tespit etmek için kullanılır.
2.make_group_NaN_and_fill_mode: Sütunda tespit edilen yanlış değerler, NaN değerlerine atanır ve her gruptaki diğer değerlerin moduyla doldurulur.
3.Fonksiyonun ana gövdesi, make_group_NaN_and_fill_mode işlevini çağıran ve sonucu ekrana yazdıran veya değişiklikleri yerinde yapacak şekilde veri çerçevesini değiştiren bir kod bloğudur.

Bu işlev, belirtilen sütunun değer aralığında olmayan yanlış değerleri tespit etmek ve onları gruba göre mod ile doldurmak için oldukça kullanışlı bir araçtır.'''

'''ENG
This code is a function that reassigns the wrong values in a numerical column for each group in a pandas DataFrame based on the mode of the valid values for that group.

If inplace is True, the function updates the DataFrame in place and prints out the original and cleaned values for the specified column for one example group before and after cleaning. If inplace is False, the function returns a copy of the cleaned DataFrame without printing any output.
'''

In [123]:
df.describe().columns

Index(['ID', 'Customer_ID', 'Month', 'Age', 'SSN', 'Annual_Income',
       'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio',
       'Credit_History_Age', 'Total_EMI_per_month', 'Amount_invested_monthly',
       'Monthly_Balance'],
      dtype='object')

In [124]:
# ID
df['ID'].nunique()

150000

In [125]:
# Month
df['Month'].value_counts()

1     12500
2     12500
3     12500
4     12500
5     12500
6     12500
7     12500
8     12500
9     12500
10    12500
11    12500
12    12500
Name: Month, dtype: int64

In [131]:
# Age
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Age')


Existing Min, Max Values:
min    14.0
max    56.0
Name: Age, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	14.0,
max:	56.0

Before Assigning Min, Max Age: ['No NaN Value']

Before Assigning Example Age:

[37.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0]
[48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 49.0]
[37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0]
[22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 23.0, 23.0, 23.0, 23.0, 23.0]
[43.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0]

After Assigning  Min, Max Age: ['No NaN Value']

After Assigning Example Age:

[37.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0]
[48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 49.0]
[37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0]
[22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 23.0, 23.0, 23.0, 23.0, 23.0]
[43.0, 44.0, 44.0, 44.0, 44.0, 44.

In [132]:
# SSN
df.SSN.value_counts(dropna=False)

NaN            8400
78735990.0       12
47161543.0       12
255398777.0      12
159517992.0      12
               ... 
414769835.0       7
695717676.0       7
803446289.0       7
74528105.0        7
642737670.0       6
Name: SSN, Length: 12501, dtype: int64

In [133]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'SSN')


Existing Min, Max Values:
min        81349.0
max    999993421.0
Name: SSN, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	81349.0,
max:	999993421.0

Before Assigning Min, Max SSN: ['No NaN Value']

Before Assigning Example SSN:

[354656948.0, 354656948.0, 354656948.0, 354656948.0, 354656948.0, 354656948.0, 354656948.0, 354656948.0, 354656948.0, 354656948.0, nan, 354656948.0]
[964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0, 964812710.0]
[802194704.0, nan, 802194704.0, 802194704.0, 802194704.0, 802194704.0, 802194704.0, 802194704.0, 802194704.0, 802194704.0, 802194704.0, 802194704.0]
[nan, nan, 891062189.0, 891062189.0, 891062189.0, 891062189.0, 891062189.0, 891062189.0, 891062189.0, 891062189.0, 891062189.0, 891062189.0]
[422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 422130011.0, 4221300

In [134]:
# Annual_Income
df.Annual_Income.value_counts(dropna=False)

109945.32      24
20867.67       24
9141.63        24
17816.75       24
40341.16       24
               ..
23467279.00     1
11586734.00     1
18324322.00     1
3449550.00      1
250738.00       1
Name: Annual_Income, Length: 13988, dtype: int64

In [135]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Annual_Income')


Existing Min, Max Values:
min        7005.93
max    24198062.00
Name: Annual_Income, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	7005.93,
max:	179987.28

Before Assigning Min, Max Annual_Income: ['No NaN Value']

Before Assigning Example Annual_Income:

[16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18, 16756.18]
[21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91, 21212.91]
[33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43, 33540.43]
[80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64, 80983.64]
[104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56, 104142.56]

After Assigning  Min, Max Annual_Income: ['No NaN Value']

After Assigning Example Annual_Income:

[1

In [136]:
# Monthly_Inhand_Salary
df.Monthly_Inhand_Salary.value_counts(dropna=False)

NaN             22500
2295.058333        22
6082.187500        22
6358.956667        21
3080.555000        21
                ...  
4305.385250         1
2028.727911         1
3065.776667         1
2486.532500         1
10699.849855        1
Name: Monthly_Inhand_Salary, Length: 13684, dtype: int64

In [137]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Inhand_Salary')


Existing Min, Max Values:
min      303.645417
max    15204.633333
Name: Monthly_Inhand_Salary, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	303.6454166666666,
max:	15204.633333333331

Before Assigning Min, Max Monthly_Inhand_Salary: ['No NaN Value']

Before Assigning Example Monthly_Inhand_Salary:

[1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, 1331.3483333333334, nan, 1331.3483333333334]
[1496.7425, 1496.7425, 1496.7425, 1496.7425, nan, 1496.7425, 1496.7425, 1496.7425, 1496.7425, 1496.7425, 1496.7425, 1496.7425]
[2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333, 2655.035833333333]
[6692.636666666666, 6692.636666666666, 6692.636666666666, 6692.636666666666, 6692.6366666666

In [138]:
# Num_Bank_Accounts
df.Num_Bank_Accounts.value_counts(dropna=False)

6       19505
7       19231
8       19152
4       18286
5       18186
        ...  
1731        1
84          1
1458        1
1448        1
278         1
Name: Num_Bank_Accounts, Length: 1183, dtype: int64

In [139]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Bank_Accounts')


Existing Min, Max Values:
min      -1
max    1798
Name: Num_Bank_Accounts, dtype: int64

Groupby by Customer_ID's Actual min, max Values:
min:	-1,
max:	11

Before Assigning Min, Max Num_Bank_Accounts: ['No NaN Value']

Before Assigning Example Num_Bank_Accounts:

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
[3, 3, 3, 1174, 3, 3, 3, 3, 3, 3, 3, 3]
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

After Assigning  Min, Max Num_Bank_Accounts: ['No NaN Value']

After Assigning Example Num_Bank_Accounts:

[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0]
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [140]:
# Num_Credit_Card
df_train.Num_Credit_Card.value_counts(dropna=False)

5       18459
7       16615
6       16559
4       14030
3       13277
        ...  
791         1
1118        1
657         1
640         1
679         1
Name: Num_Credit_Card, Length: 1179, dtype: int64

In [141]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Card')


Existing Min, Max Values:
min       0
max    1499
Name: Num_Credit_Card, dtype: int64

Groupby by Customer_ID's Actual min, max Values:
min:	0,
max:	11

Before Assigning Min, Max Num_Credit_Card: ['No NaN Value']

Before Assigning Example Num_Credit_Card:

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 750]
[4, 4, 4, 4, 4, 4, 4, 888, 4, 4, 4, 4]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 112]
[3, 3, 3, 3, 725, 3, 3, 3, 3, 3, 3, 3]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

After Assigning  Min, Max Num_Credit_Card: ['No NaN Value']

After Assigning Example Num_Credit_Card:

[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]


In [142]:
# Interest_Rate
df.Interest_Rate.value_counts(dropna=False)

8       7515
5       7479
6       7089
12      6828
10      6799
        ... 
4985       1
2781       1
608        1
4817       1
4252       1
Name: Interest_Rate, Length: 2394, dtype: int64

In [143]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Interest_Rate')


Existing Min, Max Values:
min       1
max    5799
Name: Interest_Rate, dtype: int64

Groupby by Customer_ID's Actual min, max Values:
min:	1,
max:	34

Before Assigning Min, Max Interest_Rate: ['No NaN Value']

Before Assigning Example Interest_Rate:

[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

After Assigning  Min, Max Interest_Rate: ['No NaN Value']

After Assigning Example Interest_Rate:

[22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0]
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0]
[15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]


In [144]:
# Num_of_Loan
df.Num_of_Loan.value_counts(dropna=False)

3       22618
2       22547
4       22111
0       16376
1       15901
        ...  
1085        1
742         1
291         1
365         1
1442        1
Name: Num_of_Loan, Length: 600, dtype: int64

In [145]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Loan')


Existing Min, Max Values:
min    -100
max    1496
Name: Num_of_Loan, dtype: int64

Groupby by Customer_ID's Actual min, max Values:
min:	0,
max:	9

Before Assigning Min, Max Num_of_Loan: ['No NaN Value']

Before Assigning Example Num_of_Loan:

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100]
[3, 3, 3, 3, -100, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

After Assigning  Min, Max Num_of_Loan: ['No NaN Value']

After Assigning Example Num_of_Loan:

[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [147]:
# Delay_from_due_date
df.Delay_from_due_date.value_counts(dropna=False)

 15    5355
 13    5185
 8     5004
 14    4949
 10    4926
       ... 
 63      90
 65      86
-5       51
 66      44
 67      29
Name: Delay_from_due_date, Length: 73, dtype: int64

In [148]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Delay_from_due_date')


Existing Min, Max Values:
min    -5
max    67
Name: Delay_from_due_date, dtype: int64

Groupby by Customer_ID's Actual min, max Values:
min:	-1,
max:	62

Before Assigning Min, Max Delay_from_due_date: ['No NaN Value']

Before Assigning Example Delay_from_due_date:

[48, 48, 48, 48, 48, 48, 48, 48, 48, 50, 48, 53]
[19, 19, 19, 19, 19, 19, 19, 16, 20, 19, 15, 19]
[25, 28, 22, 26, 26, 26, 26, 26, 26, 26, 21, 26]
[19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 21, 19]
[15, 15, 20, 16, 20, 20, 20, 24, 25, 20, 20, 20]

After Assigning  Min, Max Delay_from_due_date: ['No NaN Value']

After Assigning Example Delay_from_due_date:

[48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 50.0, 48.0, 53.0]
[19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 16.0, 20.0, 19.0, 15.0, 19.0]
[25.0, 28.0, 22.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 21.0, 26.0]
[19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 18.0, 21.0, 19.0]
[15.0, 15.0, 20.0, 16.0, 20.0, 20.0, 20.0, 24.0, 25.0, 20.0, 20.0, 20.0]


In [149]:
# Num_of_Delayed_Payment
df.Num_of_Delayed_Payment.value_counts(dropna=False)

NaN       10500
19.0       8188
17.0       8048
16.0       7949
15.0       7911
          ...  
3881.0        1
2728.0        1
1952.0        1
3580.0        1
2034.0        1
Name: Num_of_Delayed_Payment, Length: 1015, dtype: int64

In [150]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_of_Delayed_Payment')


Existing Min, Max Values:
min      -3.0
max    4399.0
Name: Num_of_Delayed_Payment, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	-2.0,
max:	25.0

Before Assigning Min, Max Num_of_Delayed_Payment: ['No NaN Value']

Before Assigning Example Num_of_Delayed_Payment:

[10.0, 12.0, 12.0, 13.0, nan, 12.0, 11.0, 12.0, 12.0, 12.0, 11.0, 12.0]
[19.0, 19.0, 19.0, 19.0, 19.0, 21.0, 20.0, 19.0, 19.0, 18.0, 19.0, 20.0]
[11.0, 11.0, 11.0, nan, 11.0, nan, 11.0, 13.0, 12.0, 11.0, 13.0, 11.0]
[18.0, 18.0, 18.0, 18.0, 18.0, nan, 20.0, 18.0, 18.0, 18.0, nan, nan]
[17.0, 16.0, 14.0, nan, 17.0, 14.0, 11.0, 14.0, 14.0, 14.0, 17.0, 12.0]

After Assigning  Min, Max Num_of_Delayed_Payment: ['No NaN Value']

After Assigning Example Num_of_Delayed_Payment:

[10.0, 12.0, 12.0, 13.0, 12.0, 12.0, 11.0, 12.0, 12.0, 12.0, 11.0, 12.0]
[19.0, 19.0, 19.0, 19.0, 19.0, 21.0, 20.0, 19.0, 19.0, 18.0, 19.0, 20.0]
[11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 13.0, 12.0, 11.0, 13.0, 11.0]
[18.0, 18.0, 18

In [151]:
# Changed_Credit_Limit
df.Changed_Credit_Limit.value_counts(dropna=False)

NaN      3150
11.50     197
11.32     189
8.22      189
7.35      181
         ... 
34.57       1
31.94       1
34.46       1
23.32       1
29.17       1
Name: Changed_Credit_Limit, Length: 4605, dtype: int64

In [152]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Changed_Credit_Limit')


Existing Min, Max Values:
min    -6.49
max    36.97
Name: Changed_Credit_Limit, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	0.5,
max:	29.98

Before Assigning Min, Max Changed_Credit_Limit: ['No NaN Value']

Before Assigning Example Changed_Credit_Limit:

[10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 17.66, 10.66]
[12.13, 5.13, 5.13, 5.13, 5.13, 2.13, 5.13, 5.13, 5.13, 5.13, 5.13, 5.13]
[14.11, 14.11, 14.11, 14.11, 14.11, 14.11, 14.11, 14.11, 14.11, 14.11, 14.11, 14.11]
[16.91, 16.91, 16.91, 16.91, 16.91, 16.91, 19.91, 16.91, 19.91, 16.91, 16.91, 16.91]
[15.28, 15.28, 15.28, 15.28, 19.28, 15.28, 15.28, 15.28, 15.28, 15.28, 15.28, 15.28]

After Assigning  Min, Max Changed_Credit_Limit: ['No NaN Value']

After Assigning Example Changed_Credit_Limit:

[10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 10.66, 17.66, 10.66]
[12.13, 5.13, 5.13, 5.13, 5.13, 2.13, 5.13, 5.13, 5.13, 5.13, 5.13, 5.13]
[14.11, 14.11, 14.11, 14.11, 14

In [153]:
# Num_Credit_Inquiries
df.Num_Credit_Inquiries.value_counts(dropna=False)

4.0       15673
6.0       12486
3.0       12356
7.0       12353
8.0       11788
          ...  
1243.0        1
540.0         1
2150.0        1
1324.0        1
352.0         1
Name: Num_Credit_Inquiries, Length: 1608, dtype: int64

In [154]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Num_Credit_Inquiries')


Existing Min, Max Values:
min       0.0
max    2597.0
Name: Num_Credit_Inquiries, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	0.0,
max:	17.0

Before Assigning Min, Max Num_Credit_Inquiries: ['No NaN Value']

Before Assigning Example Num_Credit_Inquiries:

[nan, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 13.0, 13.0, 13.0, 13.0]
[1.0, 1.0, 1.0, 1.0, 1196.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 5.0]
[6.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 848.0]
[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 11.0, 11.0, 11.0, 11.0]
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 692.0, 10.0, 10.0, 10.0, 10.0]

After Assigning  Min, Max Num_Credit_Inquiries: ['No NaN Value']

After Assigning Example Num_Credit_Inquiries:

[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 13.0, 13.0, 13.0, 13.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 5.0]
[6.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 11.0, 11.0, 11.0, 11.0]
[6.0, 

In [155]:
# Outstanding_Debt
df.Outstanding_Debt.value_counts(dropna=False)

1109.03    36
1151.70    36
1360.45    36
460.46     36
1058.13    24
           ..
4230.04    12
641.99     12
98.61      12
2614.48    12
502.38     12
Name: Outstanding_Debt, Length: 12203, dtype: int64

In [156]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Outstanding_Debt')


Existing Min, Max Values:
min       0.23
max    4998.07
Name: Outstanding_Debt, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	0.23,
max:	4998.07

Before Assigning Min, Max Outstanding_Debt: ['No NaN Value']

Before Assigning Example Outstanding_Debt:

[1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73]
[993.15, 993.15, 993.15, 993.15, 993.15, 993.15, 993.15, 993.15, 993.15, 993.15, 993.15, 993.15]
[1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97, 1138.97]
[982.44, 982.44, 982.44, 982.44, 982.44, 982.44, 982.44, 982.44, 982.44, 982.44, 982.44, 982.44]
[1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8, 1371.8]

After Assigning  Min, Max Outstanding_Debt: ['No NaN Value']

After Assigning Example Outstanding_Debt:

[1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73, 1941.73]

In [157]:
# Credit_Utilization_Ratio
df.Credit_Utilization_Ratio.value_counts(dropna=False)

26.822620    1
39.377219    1
40.565631    1
41.255522    1
33.638208    1
            ..
26.650258    1
23.864244    1
29.638130    1
31.875399    1
34.108530    1
Name: Credit_Utilization_Ratio, Length: 150000, dtype: int64

In [159]:
df.Credit_Utilization_Ratio.isna().sum()#there is 0 null

0

In [160]:
# Credit_History_Age
df.Credit_History_Age.value_counts(dropna=False)

NaN      13500
215.0      628
220.0      621
219.0      617
237.0      615
         ...  
3.0         20
2.0         15
407.0       15
408.0       14
1.0          2
Name: Credit_History_Age, Length: 409, dtype: int64

In [162]:
df['Credit_History_Age'] = df.groupby('Customer_ID', group_keys=False)['Credit_History_Age'].apply(lambda x: x.interpolate().bfill().ffill())

In [163]:
# Total_EMI_per_month
df.Total_EMI_per_month.value_counts(dropna=False)

0.000000        15615
49.574949          12
16.528703          12
64.443403          12
331.719510         12
                ...  
10404.000000        1
15457.000000        1
59578.000000        1
43183.000000        1
33013.000000        1
Name: Total_EMI_per_month, Length: 16960, dtype: int64

In [164]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Total_EMI_per_month')


Existing Min, Max Values:
min        0.0
max    82398.0
Name: Total_EMI_per_month, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	0.0,
max:	1779.1032538262775

Before Assigning Min, Max Total_EMI_per_month: ['No NaN Value']

Before Assigning Example Total_EMI_per_month:

[27.44208910654816, 27.44208910654816, 27.44208910654816, 27.44208910654816, 27.44208910654816, 27.44208910654816, 27.44208910654816, 32972.0, 27.44208910654816, 27.44208910654816, 27.44208910654816, 27.44208910654816]
[45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675, 45.74570037068675]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[188.45659522353543, 188.45659522353543, 188.45659522353543, 188.45659522353543, 188.45659522353543, 188.45659522353543, 188.45659522353543, 188.45659522353543, 188.45659522353543, 188.4565952235

In [165]:
# Amount_invested_monthly
df.Amount_invested_monthly.value_counts(dropna=False)

NaN             6750
10000.000000    6480
0.000000         275
80.415295          1
16.532189          1
                ... 
209.172746         1
333.014809         1
274.687129         1
220.581212         1
220.457878         1
Name: Amount_invested_monthly, Length: 136498, dtype: int64

In [166]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Amount_invested_monthly')


Existing Min, Max Values:
min        0.0
max    10000.0
Name: Amount_invested_monthly, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	0.0,
max:	10000.0

Before Assigning Min, Max Amount_invested_monthly: ['No NaN Value']

Before Assigning Example Amount_invested_monthly:

[45.30106826949194, 90.07842318605292, 61.73271462991577, 56.4949816634165, 51.726244263612, 60.82828780912217, 95.6486475869488, 66.71824831147686, 116.54410553197071, 79.08087443658461, 93.81567846791012, 168.43584020560507]
[30.373471914127194, 44.31995483866171, nan, 62.81248610216079, 148.30956484525, 10000.0, 59.30896122323683, 118.54244614778156, 104.33987358576769, 55.45176127748453, nan, nan]
[118.8806978910128, nan, 337.1997406214716, 170.8689598433119, 74.1980692925824, 105.4573791889266, 197.85037247610126, 165.20443547590202, 50.930150099663045, 86.4414839594447, 351.92301846584564, 55.62866930298164]
[149.37725143584314, 146.97809602243544, 101.12020124815511, 492.3974911743151, 2

In [167]:
# Monthly_Balance
df.Monthly_Balance.value_counts(dropna=False)

 NaN             1762
-3.333333e+26      15
 6.001125e+02       1
 1.205425e+02       1
 7.840175e+02       1
                 ... 
 2.799825e+02       1
 2.606716e+02       1
 2.504915e+02       1
 2.438753e+02       1
 3.603797e+02       1
Name: Monthly_Balance, Length: 148225, dtype: int64

In [168]:
Reassign_Group_Numeric_Wrong_Values(df, 'Customer_ID', 'Monthly_Balance')


Existing Min, Max Values:
min   -3.333333e+26
max    1.606518e+03
Name: Monthly_Balance, dtype: float64

Groupby by Customer_ID's Actual min, max Values:
min:	-3.333333333333333e+26,
max:	1183.9306960885192

Before Assigning Min, Max Monthly_Balance: ['No NaN Value']

Before Assigning Example Monthly_Balance:

[310.39167595729333, 295.61432104073225, 333.9600295968694, 309.1977625633686, 323.9664999631732, 334.864456417663, 280.04409663983637, 328.97449591530835, 279.14863869481445, 306.61186979020056, 301.87706575887506, 227.25690402118008]
[323.5550777151861, 309.6085947906515, 265.4874646531188, 291.11606352715245, 245.61898478406326, 239.46481499922072, 334.61958840607645, 275.3861034815317, 279.58867604354555, 318.4767883518287, 260.7342254927612, 307.57591763672826]
[406.62288544232047, 350.39351957834924, 218.30384271186168, 354.63462349002145, 441.30551404075095, 420.04620414440666, 337.6532108572321, 370.2991478574313, 474.5734332336703, 439.0620993738886, 203.58056486748768,

### Df cleaning is ended